# **EDA**

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('Stock Price Analysis').getOrCreate()

In [22]:
stock=spark.read.csv("/content/drive/MyDrive/stock market analysis-spark",header=True)

In [8]:
stock.show()

+------+----------+----------+-------+--------+--------+--------+
|Ticker|      Date|Close/Last| Volume|    Open|    High|     Low|
+------+----------+----------+-------+--------+--------+--------+
| BRK-B|05/31/2023|  $321.08 |6175417|$321.12 |$322.41 |$319.39 |
| BRK-B|05/30/2023|  $322.19 |3232461|$321.86 |$322.47 |$319.00 |
| BRK-B|05/26/2023|  $320.60 |3229873|$320.44 |$322.63 |$319.67 |
| BRK-B|05/25/2023|  $319.02 |4251935|$320.56 |$320.56 |$317.71 |
| BRK-B|05/24/2023|  $320.20 |3075393|$322.71 |$323.00 |$319.56 |
| BRK-B|05/23/2023|  $323.11 |4031342|$328.19 |$329.27 |$322.97 |
| BRK-B|05/22/2023|  $329.13 |2763422|$330.75 |$331.49 |$328.35 |
| BRK-B|05/19/2023|  $330.39 |4323538|$331.00 |$333.94 |$329.12 |
| BRK-B|05/18/2023|  $329.76 |2808329|$326.87 |$329.98 |$325.85 |
| BRK-B|05/17/2023|  $327.39 |3047626|$325.02 |$328.26 |$324.82 |
| BRK-B|05/16/2023|  $323.75 |2139996|$322.46 |$324.69 |$322.36 |
| BRK-B|05/15/2023|  $323.53 |2191609|$322.89 |$323.83 |$320.13 |
| BRK-B|05

In [9]:
stock.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)



In [11]:
stock.select(['Ticker','Date','Open']).show(5)

+------+----------+--------+
|Ticker|      Date|    Open|
+------+----------+--------+
| BRK-B|05/31/2023|$321.12 |
| BRK-B|05/30/2023|$321.86 |
| BRK-B|05/26/2023|$320.44 |
| BRK-B|05/25/2023|$320.56 |
| BRK-B|05/24/2023|$322.71 |
+------+----------+--------+
only showing top 5 rows



In [13]:
stock.filter(stock.Ticker=="MSFT").show(5)

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  MSFT|05/31/2023|  $328.39 |45950550|$332.29 |$335.94 |$327.33 |
|  MSFT|05/30/2023|  $331.21 |29503070|$335.23 |$335.74 |$330.52 |
|  MSFT|05/26/2023|  $332.89 |36630630|$324.02 |$333.40 |$323.88 |
|  MSFT|05/25/2023|  $325.92 |43301740|$323.24 |$326.90 |$320.00 |
|  MSFT|05/24/2023|  $313.85 |23384890|$314.73 |$316.50 |$312.61 |
+------+----------+----------+--------+--------+--------+--------+
only showing top 5 rows



In [15]:
stock.filter((stock.Ticker=="MSFT") & (stock.Date=="05/31/2023")).show(5)

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  MSFT|05/31/2023|  $328.39 |45950550|$332.29 |$335.94 |$327.33 |
+------+----------+----------+--------+--------+--------+--------+



In [30]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
from datetime import datetime
from pyspark.sql.types import FloatType


In [18]:
date_parser=udf(lambda date: datetime.strptime(date,'%m/%d/%Y'),DateType())

In [23]:
stock=stock.withColumn("ParseDate",date_parser(stock.Date))

In [24]:
stock.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- ParseDate: date (nullable = true)



In [31]:
def prices(value):
  if isinstance(value,str):
    return float(value.strip("$"))
  elif isinstance(value,int) or isinstance(value,float):
    return value
  else:
    return None
prices=udf(prices,FloatType())

In [33]:
stock=stock.withColumn("Open",prices(stock.Open))
stock=stock.withColumn("High",prices(stock.High))
stock=stock.withColumn("Low",prices(stock.Low))
stock=stock.withColumn("Close",prices(stock['Close/Last']))

In [34]:
stock.show(5)

+------+----------+----------+-------+------+------+------+----------+------+
|Ticker|      Date|Close/Last| Volume|  Open|  High|   Low| ParseDate| Close|
+------+----------+----------+-------+------+------+------+----------+------+
| BRK-B|05/31/2023|  $321.08 |6175417|321.12|322.41|319.39|2023-05-31|321.08|
| BRK-B|05/30/2023|  $322.19 |3232461|321.86|322.47| 319.0|2023-05-30|322.19|
| BRK-B|05/26/2023|  $320.60 |3229873|320.44|322.63|319.67|2023-05-26| 320.6|
| BRK-B|05/25/2023|  $319.02 |4251935|320.56|320.56|317.71|2023-05-25|319.02|
| BRK-B|05/24/2023|  $320.20 |3075393|322.71| 323.0|319.56|2023-05-24| 320.2|
+------+----------+----------+-------+------+------+------+----------+------+
only showing top 5 rows



In [35]:
from pyspark.sql.types import IntegerType
int_parser=udf(lambda volume: int(volume),IntegerType())
stock=stock.withColumn("Volume",int_parser(stock.Volume))

In [37]:
stock.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- ParseDate: date (nullable = true)
 |-- Close: float (nullable = true)



In [38]:
stock1=stock.select(['Ticker','ParseDate','Open','High','Low','Close','Volume'])

In [39]:
stock1.show(5)

+------+----------+------+------+------+------+-------+
|Ticker| ParseDate|  Open|  High|   Low| Close| Volume|
+------+----------+------+------+------+------+-------+
| BRK-B|2023-05-31|321.12|322.41|319.39|321.08|6175417|
| BRK-B|2023-05-30|321.86|322.47| 319.0|322.19|3232461|
| BRK-B|2023-05-26|320.44|322.63|319.67| 320.6|3229873|
| BRK-B|2023-05-25|320.56|320.56|317.71|319.02|4251935|
| BRK-B|2023-05-24|322.71| 323.0|319.56| 320.2|3075393|
+------+----------+------+------+------+------+-------+
only showing top 5 rows



In [40]:
stock1.describe(['Volume','Open','Close','Low','High']).show()

+-------+--------------------+------------------+------------------+------------------+------------------+
|summary|              Volume|              Open|             Close|               Low|              High|
+-------+--------------------+------------------+------------------+------------------+------------------+
|  count|               15108|             15108|             15108|             15108|             15108|
|   mean|5.1868408793685466E7|180.09656566181036| 180.1256089860054| 177.9982781513109| 182.1253348687101|
| stddev| 5.496484129953464E7|101.16125813324383|101.14891782168543|100.26590135955234|101.96625521621753|
|    min|              961133|             12.07|             11.93|              11.8|             12.45|
|    max|           914080943|            479.22|            477.71|            476.06|            479.98|
+-------+--------------------+------------------+------------------+------------------+------------------+



# **Basic Analysis**

In [43]:
stock1.groupBy('Ticker').max('Open').withColumnRenamed('max(Open)','Max Price').show()

+------+---------+
|Ticker|Max Price|
+------+---------+
| BRK-B|   361.39|
|  AAPL|   182.63|
|  META|   381.68|
|  TSLA|   411.47|
|  AMZN|    187.2|
|  MSFT|   344.62|
|   TSM|   141.61|
|   QQQ|   405.57|
|     V|   250.05|
| GOOGL|   151.25|
|   SPY|   479.22|
|  NVDA|   405.95|
+------+---------+



In [44]:
import pyspark.sql.functions as func
stock1.groupBy('Ticker').agg(func.max('Open').alias('Max Price')).show()

+------+---------+
|Ticker|Max Price|
+------+---------+
| BRK-B|   361.39|
|  AAPL|   182.63|
|  META|   381.68|
|  TSLA|   411.47|
|  AMZN|    187.2|
|  MSFT|   344.62|
|   TSM|   141.61|
|   QQQ|   405.57|
|     V|   250.05|
| GOOGL|   151.25|
|   SPY|   479.22|
|  NVDA|   405.95|
+------+---------+



In [45]:
stock1.groupBy('Ticker').agg(func.max('Open').alias('Max Price'),
                             func.sum('Volume').alias('Total Volume')).show()

+------+---------+------------+
|Ticker|Max Price|Total Volume|
+------+---------+------------+
| BRK-B|   361.39|  5862401321|
|  AAPL|   182.63|139310061360|
|  META|   381.68| 30148848043|
|  TSLA|   411.47|171802975076|
|  AMZN|    187.2|104503287430|
|  MSFT|   344.62| 37976660472|
|   TSM|   141.61| 12506470104|
|   QQQ|   405.57| 60437153773|
|     V|   250.05| 10410997871|
| GOOGL|   151.25| 43956560981|
|   SPY|   479.22|107925285300|
|  NVDA|   405.95| 58787218324|
+------+---------+------------+



In [46]:
stock1=stock1.withColumn("Year",func.year(stock1.ParseDate))
stock1=stock1.withColumn("Month",func.month(stock1.ParseDate))
stock1=stock1.withColumn("Day",func.dayofmonth(stock1.ParseDate))
stock1=stock1.withColumn("Week",func.weekofyear(stock1.ParseDate))

In [47]:
stock1.show(5)

+------+----------+------+------+------+------+-------+----+-----+---+----+
|Ticker| ParseDate|  Open|  High|   Low| Close| Volume|Year|Month|Day|Week|
+------+----------+------+------+------+------+-------+----+-----+---+----+
| BRK-B|2023-05-31|321.12|322.41|319.39|321.08|6175417|2023|    5| 31|  22|
| BRK-B|2023-05-30|321.86|322.47| 319.0|322.19|3232461|2023|    5| 30|  22|
| BRK-B|2023-05-26|320.44|322.63|319.67| 320.6|3229873|2023|    5| 26|  21|
| BRK-B|2023-05-25|320.56|320.56|317.71|319.02|4251935|2023|    5| 25|  21|
| BRK-B|2023-05-24|322.71| 323.0|319.56| 320.2|3075393|2023|    5| 24|  21|
+------+----------+------+------+------+------+-------+----+-----+---+----+
only showing top 5 rows



In [58]:
yearly=stock1.groupBy(['Ticker','Year']).agg(func.max('Open').alias('High in a year'),func.min('Open').alias('Low in a year'))

In [59]:
monthly=stock1.groupBy(['Ticker','Year','Month']).agg(func.max('Open').alias('High in a month'),func.min('Open').alias('Low in a month'))

In [60]:
weekly=stock1.groupBy(['Ticker','Year','Week']).agg(func.max('Open').alias('High in a week'),func.min('Open').alias('Low in a week'))

In [61]:
weekly.show()

+------+----+----+--------------+-------------+
|Ticker|Year|Week|High in a week|Low in a week|
+------+----+----+--------------+-------------+
| BRK-B|2022|  14|         352.0|       341.17|
| BRK-B|2022|  10|        326.59|       322.49|
| BRK-B|2021|  14|        264.22|       260.02|
|  META|2022|  43|        131.68|        97.98|
|  META|2020|   6|        212.51|       203.44|
|  TSLA|2022|  20|        255.72|       235.67|
|  TSLA|2020|  19|         52.92|        46.73|
|  TSLA|2020|  16|         51.49|        39.34|
|  TSLA|2018|  39|         20.86|        18.02|
| BRK-B|2018|  48|        217.23|        209.3|
|  MSFT|2022|   6|        309.87|       301.25|
|  MSFT|2021|   2|        218.47|       213.52|
|  META|2022|  40|        140.49|       136.76|
|  AAPL|2020|  27|         91.96|        88.31|
| BRK-B|2020|  19|        180.05|        173.4|
|  MSFT|2020|   1|        158.78|       158.32|
|  META|2020|  36|        298.88|       287.25|
|  AAPL|2021|  25|        134.45|       

In [62]:
weekly.withColumn("Spread",weekly['High in a week']-weekly['Low in a week']).show()

+------+----+----+--------------+-------------+----------+
|Ticker|Year|Week|High in a week|Low in a week|    Spread|
+------+----+----+--------------+-------------+----------+
| BRK-B|2022|  14|         352.0|       341.17| 10.829987|
| BRK-B|2022|  10|        326.59|       322.49|  4.100006|
| BRK-B|2021|  14|        264.22|       260.02|  4.200012|
|  META|2022|  43|        131.68|        97.98|  33.69999|
|  META|2020|   6|        212.51|       203.44|  9.069992|
|  TSLA|2022|  20|        255.72|       235.67| 20.050003|
|  TSLA|2020|  19|         52.92|        46.73| 6.1899986|
|  TSLA|2020|  16|         51.49|        39.34| 12.150002|
|  TSLA|2018|  39|         20.86|        18.02| 2.8400002|
| BRK-B|2018|  48|        217.23|        209.3| 7.9299927|
|  MSFT|2022|   6|        309.87|       301.25|  8.619995|
|  MSFT|2021|   2|        218.47|       213.52|  4.949997|
|  META|2022|  40|        140.49|       136.76|  3.730011|
|  AAPL|2020|  27|         91.96|        88.31| 3.650001

# **Join & SQL**

In [97]:
combined=stock1.join(yearly,(stock1.Ticker==yearly.Ticker)&(stock1.Year==yearly.Year),'inner').drop(yearly.Ticker,yearly.Year)

In [98]:
combined.show()

+----------+------+------+------+------+-------+-----+---+----+------+----+--------------+-------------+
| ParseDate|  Open|  High|   Low| Close| Volume|Month|Day|Week|Ticker|Year|High in a year|Low in a year|
+----------+------+------+------+------+-------+-----+---+----+------+----+--------------+-------------+
|2023-05-31|321.12|322.41|319.39|321.08|6175417|    5| 31|  22| BRK-B|2023|         331.0|       294.68|
|2023-05-30|321.86|322.47| 319.0|322.19|3232461|    5| 30|  22| BRK-B|2023|         331.0|       294.68|
|2023-05-26|320.44|322.63|319.67| 320.6|3229873|    5| 26|  21| BRK-B|2023|         331.0|       294.68|
|2023-05-25|320.56|320.56|317.71|319.02|4251935|    5| 25|  21| BRK-B|2023|         331.0|       294.68|
|2023-05-24|322.71| 323.0|319.56| 320.2|3075393|    5| 24|  21| BRK-B|2023|         331.0|       294.68|
|2023-05-23|328.19|329.27|322.97|323.11|4031342|    5| 23|  21| BRK-B|2023|         331.0|       294.68|
|2023-05-22|330.75|331.49|328.35|329.13|2763422|    5| 

In [99]:
combined=combined.join(weekly,(combined.Ticker==weekly.Ticker)&(combined.Year==weekly.Year) & (combined.Week==weekly.Week) ,'inner').drop(combined.Ticker,weekly.Year,weekly.Week)

In [100]:
combined.show()

+----------+------+------+------+------+-------+-----+---+----+--------------+-------------+------+----+--------------+-------------+
| ParseDate|  Open|  High|   Low| Close| Volume|Month|Day|Year|High in a year|Low in a year|Ticker|Week|High in a week|Low in a week|
+----------+------+------+------+------+-------+-----+---+----+--------------+-------------+------+----+--------------+-------------+
|2023-05-31|321.12|322.41|319.39|321.08|6175417|    5| 31|2023|         331.0|       294.68| BRK-B|  22|        321.86|       321.12|
|2023-05-30|321.86|322.47| 319.0|322.19|3232461|    5| 30|2023|         331.0|       294.68| BRK-B|  22|        321.86|       321.12|
|2023-05-26|320.44|322.63|319.67| 320.6|3229873|    5| 26|2023|         331.0|       294.68| BRK-B|  21|        330.75|       320.44|
|2023-05-25|320.56|320.56|317.71|319.02|4251935|    5| 25|2023|         331.0|       294.68| BRK-B|  21|        330.75|       320.44|
|2023-05-24|322.71| 323.0|319.56| 320.2|3075393|    5| 24|2023

In [101]:
combined=combined.join(monthly,['Ticker','Year','Month'])

In [102]:
combined.show()

+------+----+-----+----------+------+------+------+------+-------+---+--------------+-------------+----+--------------+-------------+---------------+--------------+
|Ticker|Year|Month| ParseDate|  Open|  High|   Low| Close| Volume|Day|High in a year|Low in a year|Week|High in a week|Low in a week|High in a month|Low in a month|
+------+----+-----+----------+------+------+------+------+-------+---+--------------+-------------+----+--------------+-------------+---------------+--------------+
| BRK-B|2023|    5|2023-05-31|321.12|322.41|319.39|321.08|6175417| 31|         331.0|       294.68|  22|        321.86|       321.12|          331.0|        320.44|
| BRK-B|2023|    5|2023-05-30|321.86|322.47| 319.0|322.19|3232461| 30|         331.0|       294.68|  22|        321.86|       321.12|          331.0|        320.44|
| BRK-B|2023|    5|2023-05-26|320.44|322.63|319.67| 320.6|3229873| 26|         331.0|       294.68|  21|        330.75|       320.44|          331.0|        320.44|
| BRK-B|20

In [103]:
combined.columns

['Ticker',
 'Year',
 'Month',
 'ParseDate',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'Day',
 'High in a year',
 'Low in a year',
 'Week',
 'High in a week',
 'Low in a week',
 'High in a month',
 'Low in a month']

In [104]:
final=combined.select(['Ticker','Year','Month','Day','Week','Volume','Open','High','Low','Close','High in a year','Low in a year','High in a week','Low in a week','High in a month','Low in a month'])

In [105]:
final.show()

+------+----+-----+---+----+-------+------+------+------+------+--------------+-------------+--------------+-------------+---------------+--------------+
|Ticker|Year|Month|Day|Week| Volume|  Open|  High|   Low| Close|High in a year|Low in a year|High in a week|Low in a week|High in a month|Low in a month|
+------+----+-----+---+----+-------+------+------+------+------+--------------+-------------+--------------+-------------+---------------+--------------+
| BRK-B|2023|    5| 31|  22|6175417|321.12|322.41|319.39|321.08|         331.0|       294.68|        321.86|       321.12|          331.0|        320.44|
| BRK-B|2023|    5| 30|  22|3232461|321.86|322.47| 319.0|322.19|         331.0|       294.68|        321.86|       321.12|          331.0|        320.44|
| BRK-B|2023|    5| 26|  21|3229873|320.44|322.63|319.67| 320.6|         331.0|       294.68|        330.75|       320.44|          331.0|        320.44|
| BRK-B|2023|    5| 25|  21|4251935|320.56|320.56|317.71|319.02|         331

In [108]:
final.createOrReplaceTempView('finalstock')

In [109]:
spark.sql("select * from finalstock where Ticker='MSFT' and Year='2023'").show(5)

+------+----+-----+---+----+--------+------+------+------+------+--------------+-------------+--------------+-------------+---------------+--------------+
|Ticker|Year|Month|Day|Week|  Volume|  Open|  High|   Low| Close|High in a year|Low in a year|High in a week|Low in a week|High in a month|Low in a month|
+------+----+-----+---+----+--------+------+------+------+------+--------------+-------------+--------------+-------------+---------------+--------------+
|  MSFT|2023|    5| 31|  22|45950550|332.29|335.94|327.33|328.39|        335.23|        223.0|        335.23|       332.29|         335.23|        305.72|
|  MSFT|2023|    5| 30|  22|29503070|335.23|335.74|330.52|331.21|        335.23|        223.0|        335.23|       332.29|         335.23|        305.72|
|  MSFT|2023|    5| 26|  21|36630630|324.02| 333.4|323.88|332.89|        335.23|        223.0|        324.02|       314.73|         335.23|        305.72|
|  MSFT|2023|    5| 25|  21|43301740|323.24| 326.9| 320.0|325.92|     

# **Window Functions**

In [112]:
from pyspark.sql.window import Window

In [111]:
df=stock1.select(['Ticker','ParseDate','Open'])

In [113]:
lag1day=Window.partitionBy('Ticker').orderBy('ParseDate')

For getting the corresponding previous day's price

In [114]:
df.withColumn('lag1day',func.lag('Open',1).over(lag1day)).show()

+------+----------+-----+-------+
|Ticker| ParseDate| Open|lag1day|
+------+----------+-----+-------+
|  AAPL|2018-05-31|46.81|   NULL|
|  AAPL|2018-06-01| 47.0|  46.81|
|  AAPL|2018-06-04|47.91|   47.0|
|  AAPL|2018-06-05|48.27|  47.91|
|  AAPL|2018-06-06|48.41|  48.27|
|  AAPL|2018-06-07|48.54|  48.41|
|  AAPL|2018-06-08|47.79|  48.54|
|  AAPL|2018-06-11|47.84|  47.79|
|  AAPL|2018-06-12|47.85|  47.84|
|  AAPL|2018-06-13|48.11|  47.85|
|  AAPL|2018-06-14|47.89|  48.11|
|  AAPL|2018-06-15|47.51|  47.89|
|  AAPL|2018-06-18|46.97|  47.51|
|  AAPL|2018-06-19|46.29|  46.97|
|  AAPL|2018-06-20|46.59|  46.29|
|  AAPL|2018-06-21|46.81|  46.59|
|  AAPL|2018-06-22|46.53|  46.81|
|  AAPL|2018-06-25|45.85|  46.53|
|  AAPL|2018-06-26|45.75|  45.85|
|  AAPL|2018-06-27|46.31|  45.75|
+------+----------+-----+-------+
only showing top 20 rows



In [115]:
movingavg=Window.partitionBy('Ticker').orderBy('ParseDate').rowsBetween(-10,0)

In [117]:
(df.withColumn('movingavg',func.avg('Open').over(movingavg))
  .withColumn('movingavg',func.round('movingavg',2))).show()

+------+----------+-----+---------+
|Ticker| ParseDate| Open|movingavg|
+------+----------+-----+---------+
|  AAPL|2018-05-31|46.81|    46.81|
|  AAPL|2018-06-01| 47.0|    46.91|
|  AAPL|2018-06-04|47.91|    47.24|
|  AAPL|2018-06-05|48.27|     47.5|
|  AAPL|2018-06-06|48.41|    47.68|
|  AAPL|2018-06-07|48.54|    47.82|
|  AAPL|2018-06-08|47.79|    47.82|
|  AAPL|2018-06-11|47.84|    47.82|
|  AAPL|2018-06-12|47.85|    47.82|
|  AAPL|2018-06-13|48.11|    47.85|
|  AAPL|2018-06-14|47.89|    47.86|
|  AAPL|2018-06-15|47.51|    47.92|
|  AAPL|2018-06-18|46.97|    47.92|
|  AAPL|2018-06-19|46.29|    47.77|
|  AAPL|2018-06-20|46.59|    47.62|
|  AAPL|2018-06-21|46.81|    47.47|
|  AAPL|2018-06-22|46.53|    47.29|
|  AAPL|2018-06-25|45.85|    47.11|
|  AAPL|2018-06-26|45.75|    46.92|
|  AAPL|2018-06-27|46.31|    46.78|
+------+----------+-----+---------+
only showing top 20 rows



In [128]:
maxstock=Window.partitionBy('Ticker').orderBy(func.desc('Open'))

In [133]:
result=df.withColumn('maxstock',func.row_number().over(maxstock)).filter('maxstock<=5')

In [134]:
result.show()

+------+----------+------+--------+
|Ticker| ParseDate|  Open|maxstock|
+------+----------+------+--------+
|  AAPL|2022-01-04|182.63|       1|
|  AAPL|2021-12-13|181.12|       2|
|  AAPL|2021-12-28|180.16|       3|
|  AAPL|2022-01-05|179.61|       4|
|  AAPL|2021-12-30|179.47|       5|
|  AMZN|2021-07-12| 187.2|       1|
|  AMZN|2021-07-09|186.13|       2|
|  AMZN|2021-07-07|185.87|       3|
|  AMZN|2021-11-19|185.63|       4|
|  AMZN|2021-07-14|185.44|       5|
| BRK-B|2022-03-29|361.39|       1|
| BRK-B|2022-03-28|360.59|       2|
| BRK-B|2022-03-31| 359.0|       3|
| BRK-B|2022-03-30|354.66|       4|
| BRK-B|2022-03-25| 353.9|       5|
| GOOGL|2022-02-02|151.25|       1|
| GOOGL|2021-11-19|149.98|       2|
| GOOGL|2021-11-08|149.83|       3|
| GOOGL|2021-11-22|149.33|       4|
| GOOGL|2021-11-09|149.23|       5|
+------+----------+------+--------+
only showing top 20 rows



In [136]:
result.write.option("header",True).partitionBy('Ticker','ParseDate').csv("result")